## Titanic - Machine Learning from Disaster
### Start here! Predict survival on the Titanic and get familiar with ML basics

Dataset Description
-------------------

### Overview

The data has been split into two groups:

-   training set (train.csv)
-   test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the "ground truth") for each passenger. Your model will be based on "features" like passengers' gender and class. You can also use [feature engineering ](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

### Data Dictionary

| Variable | Definition | Key |
|---|---|---|
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex |  |
| Age | Age in years |  |
| sibsp | # of siblings / spouses aboard the Titanic |  |
| parch | # of parents / children aboard the Titanic |  |
| ticket | Ticket number |  |
| fare | Passenger fare |  |
| cabin | Cabin number |  |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes

pclass: A proxy for socio-economic status (SES)\
1st = Upper\
2nd = Middle\
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...\
Sibling = brother, sister, stepbrother, stepsister\
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...\
Parent = mother, father\
Child = daughter, son, stepdaughter, stepson\
Some children travelled only with a nanny, therefore parch=0 for them.

### SOLUTION
---

In [1]:
import pandas as pd


In [2]:
train_data = pd.read_csv("train.csv")
test_data  = pd.read_csv("test.csv")
test_ids = test_data["PassengerId"]

In [3]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  10692 	SHAPE :  (891, 12)
Test  -> SIZE :  4598 	SHAPE :  (418, 11)


In [4]:
train_data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
699,700,0,3,"Humblen, Mr. Adolf Mathias Nicolai Olsen",male,42.0,0,0,348121,7.6500,F G63,S
784,785,0,3,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
464,465,0,3,"Maisner, Mr. Simon",male,NaN,0,0,A/S 2816,8.0500,NaN,S
676,677,0,3,"Sawyer, Mr. Frederick Charles",male,24.5,0,0,342826,8.0500,NaN,S
192,193,1,3,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,1,0,350046,7.8542,NaN,S
629,630,0,3,"O'Connell, Mr. Patrick D",male,NaN,0,0,334912,7.7333,NaN,Q
465,466,0,3,"Goncalves, Mr. Manuel Estanslas",male,38.0,0,0,SOTON/O.Q. 3101306,7.0500,NaN,S
670,671,1,2,"Brown, Mrs. Thomas William Solomon (Elizabeth ...",female,40.0,1,1,29750,39.0000,NaN,S
431,432,1,3,"Thorneycroft, Mrs. Percival (Florence Kate White)",female,NaN,1,0,376564,16.1000,NaN,S


In [5]:
test_data.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
296,1188,2,"Laroche, Miss. Louise",female,1.0,1,2,SC/Paris 2123,41.5792,NaN,C
59,951,1,"Chaudanson, Miss. Victorine",female,36.0,0,0,PC 17608,262.3750,B61,C
254,1146,3,"Wenzel, Mr. Linhart",male,32.5,0,0,345775,9.5000,NaN,S
380,1272,3,"O'Connor, Mr. Patrick",male,NaN,0,0,366713,7.7500,NaN,Q
146,1038,1,"Hilliard, Mr. Herbert Henry",male,NaN,0,0,17463,51.8625,E46,S
157,1049,3,"Lundin, Miss. Olga Elida",female,23.0,0,0,347469,7.8542,NaN,S
55,947,3,"Rice, Master. Albert",male,10.0,4,1,382652,29.1250,NaN,Q
217,1109,1,"Wick, Mr. George Dennick",male,57.0,1,1,36928,164.8667,NaN,S
352,1244,2,"Dibden, Mr. William",male,18.0,0,0,S.O.C. 14879,73.5000,NaN,S
208,1100,1,"Rosenbaum, Miss. Edith Louise",female,33.0,0,0,PC 17613,27.7208,A11,C


In [6]:
# Creating Function to Drop Columns from the DataFrame
# and Fill missing rows where data in not present

def dataset_cleaner(data):
    # Dropping Following columns as they will be unnecessary for predictions
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

    # Filling missing Values
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    
    # U --> Unknown Token
    data.Embarked.fillna("U", inplace=True)

    return data


train_data = dataset_cleaner(train_data)
test_data  = dataset_cleaner(test_data)

In [7]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  7128 	SHAPE :  (891, 8)
Test  -> SIZE :  2926 	SHAPE :  (418, 7)


In [8]:
train_data.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
840,0,3,male,20.0,0,0,7.9250,S
806,0,1,male,39.0,0,0,0.0000,S
262,0,1,male,52.0,1,1,79.6500,S
509,1,3,male,26.0,0,0,56.4958,S
174,0,1,male,56.0,0,0,30.6958,C
676,0,3,male,24.5,0,0,8.0500,S
295,0,1,male,28.0,0,0,27.7208,C
773,0,3,male,28.0,0,0,7.2250,C
414,1,3,male,44.0,0,0,7.9250,S
97,1,1,male,23.0,0,1,63.3583,C


In [9]:
test_data.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
47,3,male,27.00,0,0,7.7500,Q
201,3,male,0.33,0,2,14.4000,S
341,3,male,32.00,0,0,7.5792,S
21,3,male,9.00,0,1,3.1708,S
188,3,female,27.00,8,2,69.5500,S
242,1,male,45.00,1,1,134.5000,C
262,2,female,29.00,0,2,23.0000,S
114,1,female,63.00,1,0,221.7792,S
137,2,male,26.00,0,0,13.0000,S
71,3,male,21.00,0,0,7.8958,S


In [10]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()

def encoder(data):
    # Encoding the following Columns values to encoded integers
    cols = ["Sex", "Embarked"]

    for col in cols:
        data[col] = lab_enc.fit_transform(data[col])
    

encoder(train_data)
encoder(test_data)        

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x = train_data.drop("Survived", axis=1)
y = train_data["Survived"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [12]:
classifier = LogisticRegression(random_state=0, max_iter=3000)
classifier = classifier.fit(x_train, y_train)

pred = classifier.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score

print("Classifier Accuracy :",accuracy_score(y_test, pred))

Classifier Accuracy : 0.8134328358208955


In [14]:
submit_pred = classifier.predict(test_data)

submit_df = pd.DataFrame({"PassengerId":test_ids.values,
                        "Survived": submit_pred})

submit_df.to_csv("submission.csv", index=False)